# Predicting the Risk of Employees’ Attrition with Classification Models

By: Evelyn Peh Ting Yu, Koh Pei Ling, Song Yu Xiang, Wong Jie Peng, Guo Peng Yuan

The US department of labour mentioned that “it could cost up to 11k in direct training expenses and lost productivity to replace an experienced employee earning an annual salary of $33,000.” This statement itself implies that employee attrition is costly. When an employee leaves the organization, the company is not only losing its valuable employees, but the company also loses on the amount that it has spent to recruit and select those employees and to train them for their respective jobs. On the other hand, the organization needs to invest more and more in recruitment, training, and development of new staff to fill up their vacant positions. Due to these reasons, every organization wants to control the attrition rate and retain its employees through more satisfactory company policies and work environments. 

This project, therefore, aims to predict the likelihood of an employee leaving the company by understanding the main drivers of employee churn. Such information can be valuable in helping management take possible actions to potentially retain these employees thereby reducing the high cost incurred in employee turnover. 

### 1. Import Libraries

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

#model
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier # pip install xgboost

from imblearn.over_sampling import SMOTE # !pip install imblearn

#PyTorch
import torch.nn as nn
import torch

#Save and Load Model
import joblib
import pickle

import warnings
warnings.filterwarnings('ignore')

<hr>

### 2. Import Dataset

https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset

<b>35 attributes, 1460 observations</b> <br><br>
<b>Attributes:</b> <br>
<b>Dependent Variables:</b> Attrition <br>
<b>Independent Variables:</b>
<ul>
    <li><b><i>Basic Information (6 attributes)</i></b>: Age, Education, Education Field, Gender, Marital Status, Distance From Home
    <li><b><i>Work Information (8 attributes)</i></b>: Department, Job Role, Job Level, Over Time, Business Travel, Performance Rating, Stock Option Level,
          Job Involvement
    <li><b><i>Satisfaction (4 attributes)</i></b>: Work-Life Balance, Job Satisfaction, Relationship Satisfaction, Enviroment Satisfaction
    <li><b><i>Salary Related (5 attributes)</i></b>: Monthly Income, Monthly Rate, Daily Rate, Hourly Rate, Percent Salary Hike
    <li><b><i>Time Related (7 attributes)</i></b>: Total Working Years, Training Time Last Year, Years At Company, Years In Current Role, Years Since Last Promotion, 
           Years With Current Manager, Num Companies Worked
</ul>

In [122]:
df = pd.read_csv("dataset.csv") 
df.shape #1470 rows and 35 columns

(1470, 35)

In [123]:
df.head() #top 5 records, ensure data loaded properly

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


<hr>

### 3. Exploratory Data Analysis (EDA)

refer to "Project - Exploratory Data Analysis" Notebook

<hr>

### 4. Feature Pre-processing

###### Dropping columns with constant value and identifier columns

In [124]:
col_to_drop = ['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeNumber']

In [125]:
df.drop(columns=col_to_drop, inplace=True) 

In [126]:
df.shape #35 - 4 = 31 remaining cols 

(1470, 31)

###### Encoding the categorical type data 


In [127]:
le = LabelEncoder()

In [128]:
# Label Encoding will be used for columns with 2 or less unique values
le_count = 0
for col in df.columns[1:]:
    if df[col].dtype == 'object':
        if len(list(df[col].unique())) <= 2:
            le.fit(df[col])
            df[col] = le.transform(df[col])
            le_count += 1
print('{} columns were label encoded.'.format(le_count))

3 columns were label encoded.


In [129]:
# convert rest of categorical variable into dummy
df = pd.get_dummies(df, drop_first=True)

###### Standardisation

In [130]:
# import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 5))
HR_col = list(df.columns)
HR_col.remove('Attrition')
for col in HR_col:
    df[col] = df[col].astype(float)
    df[[col]] = scaler.fit_transform(df[[col]])
df['Attrition'] = pd.to_numeric(df['Attrition'], downcast='float')
df.head()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single
0,2.738095,1.0,3.579098,0.000000,1.25,1.666667,0.0,4.571429,3.333333,1.25,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0
1,3.690476,0.0,0.633500,1.250000,0.00,3.333333,5.0,2.214286,1.666667,1.25,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0
2,2.261905,1.0,4.549034,0.178571,1.25,5.000000,5.0,4.428571,1.666667,0.00,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,1.785714,0.0,4.617037,0.357143,3.75,5.000000,0.0,1.857143,3.333333,0.00,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0
4,1.071429,0.0,1.750179,0.178571,0.00,0.000000,5.0,0.714286,3.333333,0.00,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0


In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 45 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                1470 non-null   float64
 1   Attrition                          1470 non-null   float32
 2   DailyRate                          1470 non-null   float64
 3   DistanceFromHome                   1470 non-null   float64
 4   Education                          1470 non-null   float64
 5   EnvironmentSatisfaction            1470 non-null   float64
 6   Gender                             1470 non-null   float64
 7   HourlyRate                         1470 non-null   float64
 8   JobInvolvement                     1470 non-null   float64
 9   JobLevel                           1470 non-null   float64
 10  JobSatisfaction                    1470 non-null   float64
 11  MonthlyIncome                      1470 non-null   float

In [132]:
# Separate input features and target
y = df["Attrition"]
X = df.loc[:, df.columns != 'Attrition']

<hr>

### 5. Handling imbalance data with SMOTE

In [133]:
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)


sm = SMOTE(random_state = 0)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [134]:
y_train.value_counts()

0.0    913
1.0    913
Name: Attrition, dtype: int64

<hr>

### 6. Handling imbalance data with GAN

https://realpython.com/generative-adversarial-networks/

In [135]:
# setting up testing and training sets
X_gantrain, X_gantest, y_gantrain, y_gantest = train_test_split(X, y, test_size=0.25, random_state=123)

In [136]:
process = X_gantrain.copy()
process['target'] = y_gantrain.copy()

In [137]:
process.target.value_counts()

0.0    913
1.0    189
Name: target, dtype: int64

In [138]:
X_forgenerate = process.query("target == 1").iloc[:,:-1].values
X_forgenerate = torch.tensor(X_forgenerate).type(torch.FloatTensor)

X_non_attrition = process.query('target == 0').iloc[:,:-1].values #without target col
n_generate = X_non_attrition.shape[0] - X_forgenerate.shape[0]

print(f'Have to generate {n_generate} record of attrition = 1 to balance data')

Have to generate 724 record of attrition = 1 to balance data


In [139]:
BATCH_SIZE = 50
LR_G = 0.001           
LR_D = 0.001           
N_IDEAS = 28      # Initial idea for G generator (random inspiration)


G = nn.Sequential(                      # generator
    nn.Linear(N_IDEAS, 128),            
    nn.ReLU(),
    nn.Linear(128, 44), 
)


D = nn.Sequential(                      # discriminator
    nn.Linear(44, 128),     
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64, 1),
    nn.Sigmoid()
)

# optimizator
opt_D = torch.optim.Adam(D.parameters(),lr=LR_D)
opt_G = torch.optim.Adam(G.parameters(),lr=LR_G)


In [140]:
# GAN
for step in range(3000):
    # Randomly select real samples of defined batch size with label 1
    chosen_data = np.random.choice((X_forgenerate.shape[0]),size=(BATCH_SIZE),replace=False)
    artist_paintings = X_forgenerate[chosen_data,:]
    
    # Generate fake samples using generators
    G_ideas = torch.randn(BATCH_SIZE, N_IDEAS, requires_grad=True)           
    G_paintings = G(G_ideas)                  
    
    #training the generator
    prob_artist1 = D(G_paintings) 
    
    # loss
    G_loss = torch.mean(torch.log(1. - prob_artist1))
    opt_G.zero_grad()
    G_loss.backward()
    opt_G.step()
    

    # Training the discriminator
    prob_artist0 = D(artist_paintings)
    prob_artist1 = D(G_paintings.detach())
    
    # loss
    D_loss = - torch.mean(torch.log(prob_artist0) + torch.log(1. - prob_artist1))
    opt_D.zero_grad()
    D_loss.backward(retain_graph=True)
    opt_D.step()

In [141]:
#GAN fake data
fake_data = G(torch.randn(n_generate,N_IDEAS)).detach().numpy()

X_default = pd.DataFrame(np.concatenate([X_forgenerate,fake_data]))
X_default['target'] = 1

X_non_attrition = pd.DataFrame(X_non_attrition)
X_non_attrition['target'] = 0
train_data_gan = pd.concat([X_default,X_non_attrition])

X_gan = train_data_gan.iloc[:,:-1]
y_gan = train_data_gan.iloc[:,-1]

print(X_gan.shape,y_gan.shape)

(1826, 44) (1826,)


In [142]:
X_gan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1826 entries, 0 to 912
Data columns (total 44 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1826 non-null   float64
 1   1       1826 non-null   float64
 2   2       1826 non-null   float64
 3   3       1826 non-null   float64
 4   4       1826 non-null   float64
 5   5       1826 non-null   float64
 6   6       1826 non-null   float64
 7   7       1826 non-null   float64
 8   8       1826 non-null   float64
 9   9       1826 non-null   float64
 10  10      1826 non-null   float64
 11  11      1826 non-null   float64
 12  12      1826 non-null   float64
 13  13      1826 non-null   float64
 14  14      1826 non-null   float64
 15  15      1826 non-null   float64
 16  16      1826 non-null   float64
 17  17      1826 non-null   float64
 18  18      1826 non-null   float64
 19  19      1826 non-null   float64
 20  20      1826 non-null   float64
 21  21      1826 non-null   float64
 22  2

<hr>

### 7. Testing SMOTE and GAN on Baseline Modelling

In [143]:
# set table to table to populate with performance results
col = ['Algorithm','Train Accuracy Mean', 'Test Accuracy', 'Recall']

def runmodel(X_train, y_train):
    train_acc_results = []
    test_acc_results = []
    names = []
    
    df_results = pd.DataFrame(columns=col)
    
    i = 0
    # evaluate each model using cross-validation
    for name, model in models:
        kfold = KFold(n_splits=10)  # 10-fold cross-validation

        # cv accuracy scoring
        cv_acc_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
        train_acc_results.append(cv_acc_results)

        #test accuracy scoring
        model.fit(X_train,y_train)
        pred = model.predict(X_test)
        test_accuracy = accuracy_score(y_test, pred)
        test_acc_results.append(test_accuracy)
        
        #Confusion Matrix
        cm = confusion_matrix(pred,y_test)
        tn = cm[0][0]
        fn = cm[1][0]
        fp = cm[0][1]
        tp = cm[1][1]
        precision = round(tp/(tp+fp),4)
        recall = round(tp/(tp+fn),4)
        f1 = round((2*precision*recall)/(precision + recall),4)

        names.append(name)
        df_results.loc[i] = [name, round(cv_acc_results.mean()*100, 2), round(test_accuracy*100,2), round(recall*100,2)]

        i += 1
        
    return df_results

In [144]:
# selection of algorithms to consider and set performance measure
models = []
models.append(('Logistic Regression', LogisticRegression(random_state=345)))
models.append(('Random Forest', RandomForestClassifier(random_state=345)))
models.append(('SVM', SVC(probability=True)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('Decision Tree Classifier',DecisionTreeClassifier(random_state=345)))
models.append(('Gaussian NB', GaussianNB()))
models.append(('Adaboost', AdaBoostClassifier(random_state=345)))
models.append(("Gradientboost", GradientBoostingClassifier(random_state=345)))
models.append(("BaggingClassifier", BaggingClassifier(random_state=345)))
models.append(("ExtremeGradientBoost", XGBClassifier(random_state=345)))
models.append(("ExtraTreesClassifier", ExtraTreesClassifier(random_state=345)))

###### GAN

In [145]:
gan_result = runmodel(X_gan, y_gan)

In [146]:
gan_result.sort_values(by=['Test Accuracy', 'Recall'], ascending=False)

,Algorithm,Train Accuracy Mean,Test Accuracy,Recall
7,Gradientboost,87.81,91.30,86.36
2,SVM,87.42,91.03,77.78
6,Adaboost,85.94,90.76,67.50
1,Random Forest,88.85,90.22,92.86
10,ExtraTreesClassifier,88.74,89.40,100.00
9,ExtremeGradientBoost,87.20,89.40,64.52
8,BaggingClassifier,87.15,88.04,59.09
0,Logistic Regression,85.11,86.14,47.46
3,KNN,83.79,80.16,28.07
4,Decision Tree Classifier,80.67,78.26,23.33


###### SMOTE

In [147]:
smote_result = runmodel(X_train, y_train)

In [148]:
smote_result.sort_values(by=['Test Accuracy', 'Recall'], ascending=False)

,Algorithm,Train Accuracy Mean,Test Accuracy,Recall
7,Gradientboost,91.42,90.76,75.00
10,ExtraTreesClassifier,96.56,90.22,75.00
1,Random Forest,93.88,88.86,65.22
9,ExtremeGradientBoost,92.73,88.04,55.26
8,BaggingClassifier,90.87,86.96,50.00
6,Adaboost,89.23,85.33,44.83
2,SVM,90.59,81.79,36.99
4,Decision Tree Classifier,85.34,79.08,31.65
0,Logistic Regression,76.07,74.73,30.43
3,KNN,79.94,60.60,18.71


#### Analysis

> 

<hr>

### 8. Feature Selection

In [29]:
# selection of algorithms to consider and set performance measure
fs_models = []
fs_models.append(('Logistic Regression', LogisticRegression(random_state=345)))
fs_models.append(('Random Forest', RandomForestClassifier(random_state=345)))
fs_models.append(('SVM', SVC(probability=True)))
fs_models.append(('KNN', KNeighborsClassifier()))
fs_models.append(('Decision Tree Classifier',DecisionTreeClassifier(random_state=345)))
fs_models.append(('Gaussian NB', GaussianNB()))
fs_models.append(('Adaboost', AdaBoostClassifier(random_state=345)))
fs_models.append(("Gradientboost", GradientBoostingClassifier(random_state=345)))
fs_models.append(("BaggingClassifier", BaggingClassifier(random_state=345)))
fs_models.append(("ExtremeGradientBoost", XGBClassifier(random_state=345)))
fs_models.append(("ExtraTreesClassifier", ExtraTreesClassifier(random_state=345)))

In [30]:
def feature_selection_model(model, xtrain,ytrain, xtest):
    
    rfecv = RFECV(estimator=model, cv=StratifiedKFold(10, random_state=123, shuffle=True), scoring="accuracy")
    rfecv.fit(xtrain, ytrain)
    
    return xtrain.iloc[:, rfecv.support_], xtest.iloc[:, rfecv.support_]

In [31]:
# set table to table to populate with performance results
col = ['Algorithm','Train Accuracy Mean', 'Test Accuracy', "Recall"]

def runfsmodel(X_train_importance, y_train_importance, x_test_importance):
    train_acc_results = []
    test_acc_results = []
    names = []
    
    df_results = pd.DataFrame(columns=col)
    
    i = 0
    # evaluate each model using cross-validation
    for name, model in fs_models:
        kfold = KFold(n_splits=10)  # 10-fold cross-validation

        # cv accuracy scoring
        cv_acc_results = cross_val_score(model, X_train_importance, y_train_importance, cv=kfold, scoring='accuracy')
        train_acc_results.append(cv_acc_results)

        #test accuracy scoring
        model.fit(X_train_importance,y_train_importance)
        pred = model.predict(x_test_importance)
        test_accuracy = accuracy_score(y_test, pred)
        test_acc_results.append(test_accuracy)

        #Confusion Matrix
        cm = confusion_matrix(pred,y_test)
        tn = cm[0][0]
        fn = cm[1][0]
        fp = cm[0][1]
        tp = cm[1][1]
        precision = round(tp/(tp+fp),4)
        recall = round(tp/(tp+fn),4)
        f1 = round((2*precision*recall)/(precision + recall),4)

        names.append(name)
        df_results.loc[i] = [name, round(cv_acc_results.mean()*100, 2), round(test_accuracy*100,2), round(recall*100,2)]

        i += 1
        
    return df_results

###### Logistic Regression


In [32]:
base_lr_model = fs_models[0][1]

In [33]:
LR_fs = feature_selection_model(base_lr_model, X_train, y_train, X_test)
LR_X_train_importance = LR_fs[0]
LR_X_test_importance = LR_fs[1]

print(f'Number of columns selected: {LR_X_train_importance.shape[1]}')

Number of columns selected: 41


In [34]:
LR_fs_result = runfsmodel(LR_X_train_importance, y_train, LR_X_test_importance)

In [35]:
LR_fs_result.sort_values(by=['Test Accuracy', 'Recall'], ascending=False)

,Algorithm,Train Accuracy Mean,Test Accuracy,Recall
7,Gradientboost,91.20,89.95,67.74
10,ExtraTreesClassifier,96.34,89.67,72.73
1,Random Forest,93.71,89.40,69.57
8,BaggingClassifier,91.19,89.13,63.33
9,ExtremeGradientBoost,92.90,88.86,59.46
6,Adaboost,89.01,86.68,49.21
2,SVM,90.32,81.79,37.33
4,Decision Tree Classifier,85.67,79.08,31.65
0,Logistic Regression,76.57,74.73,30.43
3,KNN,80.33,61.41,20.62


###### Decision Tree

In [36]:
base_dt_model = fs_models[4][1]

In [37]:
DT_fs = feature_selection_model(base_dt_model, X_train, y_train, X_test)
DT_X_train_importance = DT_fs[0]
DT_X_test_importance = DT_fs[1]

print(f'Number of columns selected: {DT_X_train_importance.shape[1]}')

Number of columns selected: 14


In [38]:
DT_fs_result = runfsmodel(DT_X_train_importance, y_train, DT_X_test_importance)

In [39]:
DT_fs_result.sort_values(by=['Test Accuracy', 'Recall'], ascending=False)

,Algorithm,Train Accuracy Mean,Test Accuracy,Recall
7,Gradientboost,90.38,88.04,57.14
10,ExtraTreesClassifier,94.59,86.96,50.00
1,Random Forest,91.96,85.87,43.33
9,ExtremeGradientBoost,90.65,85.60,44.19
8,BaggingClassifier,89.06,85.33,40.62
6,Adaboost,87.59,84.78,43.10
2,SVM,83.69,79.35,34.09
4,Decision Tree Classifier,84.85,78.26,30.95
0,Logistic Regression,71.58,73.64,29.75
3,KNN,80.10,67.66,22.90


###### Random Forest

In [40]:
base_rf_model = fs_models[1][1]

In [41]:
RF_fs = feature_selection_model(base_rf_model, X_train, y_train, X_test)
RF_X_train_importance = RF_fs[0]
RF_X_test_importance = RF_fs[1]

print(f'Number of columns selected: {RF_X_train_importance.shape[1]}')

Number of columns selected: 35


In [42]:
RF_fs_result = runfsmodel(RF_X_train_importance, y_train, RF_X_test_importance)

In [43]:
RF_fs_result.sort_values(by=['Test Accuracy', 'Recall'], ascending=False)

,Algorithm,Train Accuracy Mean,Test Accuracy,Recall
10,ExtraTreesClassifier,96.28,90.22,71.43
1,Random Forest,93.28,89.67,70.83
7,Gradientboost,91.03,89.40,62.86
8,BaggingClassifier,91.08,88.32,59.26
9,ExtremeGradientBoost,92.51,88.32,56.76
6,Adaboost,89.28,86.14,46.94
2,SVM,90.48,81.52,35.29
4,Decision Tree Classifier,85.02,79.89,34.52
0,Logistic Regression,75.96,75.54,31.25
3,KNN,79.67,63.04,21.43


#### Analysis

>

<hr>

## 9. Top 3 Model Tuning

In [44]:
col = ["Accuracy_Score","Recall"]
df_performance = pd.DataFrame(columns=col)

In [45]:
#To evaluate performances of all the models
def performance(name, p, ytest):
    
    accuracy = np.round(accuracy_score(p,ytest),4)
    
    cm = confusion_matrix(p,ytest)
    tn = cm[0][0]
    fn = cm[1][0]
    fp = cm[0][1]
    tp = cm[1][1]
    precision = round(tp/(tp+fp),4)
    recall = round(tp/(tp+fn),4)
    f1 = round((2*precision*recall)/(precision + recall),4)
    
    df_performance.loc[name] = [round(accuracy*100,2),round(recall*100,2)]


###### 1. Gradient Boosting

In [46]:
gb_params = {
    'n_estimators':[100,200, 300],
    'learning_rate': [0.1,0.2,0.3],
    "max_leaf_nodes": [2, 5, 10, 20, 50, 100],
}


performance gridserach on original train dataset

In [47]:
gb_gs = GridSearchCV(GradientBoostingClassifier(),gb_params, scoring=make_scorer(accuracy_score),cv=5)
gb_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.1, 0.2, 0.3],
                         'max_leaf_nodes': [2, 5, 10, 20, 50, 100],
                         'n_estimators': [100, 200, 300]},
             scoring=make_scorer(accuracy_score))

In [48]:
print(f'Best Params: {gb_gs.best_params_}')
print(f'Best Estimator: {gb_gs.best_estimator_}')
print(f'Best Score: {gb_gs.best_score_}')

Best Params: {'learning_rate': 0.1, 'max_leaf_nodes': 5, 'n_estimators': 100}
Best Estimator: GradientBoostingClassifier(max_leaf_nodes=5)
Best Score: 0.8807934725653117


In [49]:
y_pred_gb = gb_gs.predict(X_test)

performance("GradientBoosting_Tuned",y_pred_gb, y_test)

perform gridsearch on feature selection train dataset 

In [50]:
gb_gs_fs = GridSearchCV(GradientBoostingClassifier(),gb_params, scoring=make_scorer(accuracy_score),cv=5)
gb_gs_fs.fit(RF_X_train_importance, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.1, 0.2, 0.3],
                         'max_leaf_nodes': [2, 5, 10, 20, 50, 100],
                         'n_estimators': [100, 200, 300]},
             scoring=make_scorer(accuracy_score))

In [51]:
print(f'Best Params: {gb_gs_fs.best_params_}')
print(f'Best Estimator: {gb_gs_fs.best_estimator_}')
print(f'Best Score: {gb_gs_fs.best_score_}')

Best Params: {'learning_rate': 0.2, 'max_leaf_nodes': 10, 'n_estimators': 100}
Best Estimator: GradientBoostingClassifier(learning_rate=0.2, max_leaf_nodes=10)
Best Score: 0.8802530129500712


In [52]:
y_important_pred = gb_gs_fs.predict(RF_X_test_importance)

In [53]:

performance("GradientBoosting_FS_Tuned", y_important_pred, y_test)

In [54]:
df_performance.head()

,Accuracy_Score,Recall
GradientBoosting_Tuned,89.95,68.97
GradientBoosting_FS_Tuned,89.40,63.64


###### Analysis

>

###### 2. ExtraTreesClassifier

In [55]:
et_params = {"max_depth": [None,2,6,10],
             'max_features':["sqrt", "log2", None],
             "n_estimators": [100,300,500],
             "criterion": ["gini", "entropy", "log_loss"]}

performance gridsearch on original train dataset

In [56]:
et_gs = GridSearchCV(ExtraTreesClassifier(),et_params, scoring=make_scorer(accuracy_score),cv=5)
et_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 2, 6, 10],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [100, 300, 500]},
             scoring=make_scorer(accuracy_score))

In [57]:
print(f'Best Params: {et_gs.best_params_}')
print(f'Best Estimator: {et_gs.best_estimator_}')
print(f'Best Score: {et_gs.best_score_}')

Best Params: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'n_estimators': 300}
Best Estimator: ExtraTreesClassifier(max_features='log2', n_estimators=300)
Best Score: 0.9671607156224267


In [58]:
y_pred_et = et_gs.predict(X_test)

performance("ExtraTreeClassifier_Tuned",y_pred_et, y_test)

perform gridsearch on feature selection train dataset

In [59]:
et_gs_fs = GridSearchCV(ExtraTreesClassifier(),et_params, scoring=make_scorer(accuracy_score),cv=5)
et_gs_fs.fit(RF_X_train_importance, y_train)

GridSearchCV(cv=5, estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 2, 6, 10],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [100, 300, 500]},
             scoring=make_scorer(accuracy_score))

In [60]:
print(f'Best Params: {et_gs_fs.best_params_}')
print(f'Best Estimator: {et_gs_fs.best_estimator_}')
print(f'Best Score: {et_gs_fs.best_score_}')

Best Params: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 300}
Best Estimator: ExtraTreesClassifier(n_estimators=300)
Best Score: 0.9698989445317764


In [61]:
y_important_pred = et_gs_fs.predict(RF_X_test_importance)

In [62]:
performance("ExtraTreeClassifier_FS_Tuned", y_important_pred, y_test)

In [63]:
df_performance.head()

,Accuracy_Score,Recall
GradientBoosting_Tuned,89.95,68.97
GradientBoosting_FS_Tuned,89.40,63.64
ExtraTreeClassifier_Tuned,90.22,75.00
ExtraTreeClassifier_FS_Tuned,89.95,73.91


###### 3. Random Forest

In [64]:
rf_params={
        'n_estimators':[100, 300, 500],
        'criterion':['gini','entropy'],
        'max_depth':[None, 3,5,7],
        'max_features':["sqrt", "log2", None]
       }

In [65]:
rf_gs=GridSearchCV(RandomForestClassifier(),rf_params,scoring=make_scorer(accuracy_score),cv=5)
rf_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 3, 5, 7],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [100, 300, 500]},
             scoring=make_scorer(accuracy_score))

In [66]:
print(f'Best Params: {rf_gs.best_params_}')
print(f'Best Estimator: {rf_gs.best_estimator_}')
print(f'Best Score: {rf_gs.best_score_}')

Best Params: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 500}
Best Estimator: RandomForestClassifier(n_estimators=500)
Best Score: 0.9360191631110112


In [67]:
y_pred_rf = rf_gs.predict(X_test)

performance("RandomForest_Tuned",y_pred_rf, y_test)

perform gridsearch on feature selection train dataset

In [68]:
rf_gs_fs=GridSearchCV(RandomForestClassifier(),rf_params,scoring=make_scorer(accuracy_score),cv=5)
rf_gs_fs.fit(RF_X_train_importance, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 3, 5, 7],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [100, 300, 500]},
             scoring=make_scorer(accuracy_score))

In [69]:
print(f'Best Params: {rf_gs_fs.best_params_}')
print(f'Best Estimator: {rf_gs_fs.best_estimator_}')
print(f'Best Score: {rf_gs_fs.best_score_}')

Best Params: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'n_estimators': 300}
Best Estimator: RandomForestClassifier(max_features='log2', n_estimators=300)
Best Score: 0.931095141851935


In [70]:
y_important_pred = rf_gs_fs.predict(RF_X_test_importance)

In [71]:
performance("RandomForest_FS_Tuned", y_important_pred, y_test)

In [72]:
df_performance.head(6)

,Accuracy_Score,Recall
GradientBoosting_Tuned,89.95,68.97
GradientBoosting_FS_Tuned,89.40,63.64
ExtraTreeClassifier_Tuned,90.22,75.00
ExtraTreeClassifier_FS_Tuned,89.95,73.91
RandomForest_Tuned,89.40,69.57
RandomForest_FS_Tuned,88.32,61.90


### 10. Final Evaluation

In [81]:
rename_index={'Gradientboost': 'Gradientboost_Baseline','ExtraTreesClassifier': 'ExtraTreesClassifier_Baseline','Random Forest': 'RandomForest_Baseline' }

In [82]:
smote_baseline = smote_result.sort_values(by=['Test Accuracy', 'Recall'], ascending=False).iloc[:3].set_index(['Algorithm']).drop(['Train Accuracy Mean'], axis=1)

In [83]:
smote_baseline.rename(columns={'Test Accuracy': 'Accuracy_Score'},index = rename_index, inplace=True)

In [84]:
rename_index={'Gradientboost': 'Gradientboost_FS_Baseline','ExtraTreesClassifier': 'ExtraTreesClassifier_FS_Baseline','Random Forest': 'RandomForest_FS_Baseline' }

In [85]:
df_rs_result = RF_fs_result.sort_values(by=['Test Accuracy', 'Recall'], ascending=False).iloc[:3].set_index(['Algorithm']).drop(['Train Accuracy Mean'], axis=1)

In [86]:
df_rs_result.rename(columns={'Test Accuracy': 'Accuracy_Score'}, index = rename_index, inplace=True)

In [87]:
pd.concat([df_performance, smote_baseline,df_rs_result]).sort_index()

,Accuracy_Score,Recall
ExtraTreeClassifier_FS_Tuned,89.95,73.91
ExtraTreeClassifier_Tuned,90.22,75.00
ExtraTreesClassifier_Baseline,90.22,75.00
ExtraTreesClassifier_FS_Baseline,90.22,71.43
GradientBoosting_FS_Tuned,89.40,63.64
GradientBoosting_Tuned,89.95,68.97
Gradientboost_Baseline,90.76,75.00
Gradientboost_FS_Baseline,89.40,62.86
RandomForest_Baseline,88.86,65.22
RandomForest_FS_Baseline,89.67,70.83


<hr>

### 11. Save Model

In [149]:
#baseline model w original dataset
filename_gb_baseline = 'finalized_baseline_gbmodel.joblib'
filename_et_baseline = 'finalized_baseline_etmodel.joblib'
filename_rf_baseline = 'finalized_baseline_rfmodel.joblib'

#baseline model w fs dataset
filename_gb_fs_baseline = 'finalized_baseline_fs_gbmodel.joblib'
filename_et_fs_baseline = 'finalized_baseline_fs_etmodel.joblib'
filename_rf_fs_baseline = 'finalized_baseline_fs_rfmodel.joblib'

#tuned w original datset
filename_gb = 'finalized_tuned_gbmodel.joblib'
filename_et = 'finalized_tuned_etmodel.joblib'
filename_rf = 'finalized_tuned_rfmodel.joblib'

#tuned w fs dataset
filename_gb_fs = 'finalized_tunedfs_gbmodel.joblib'
filename_et_fs = 'finalized_tunedfs_etmodel.joblib'
filename_rf_fs = 'finalized_tunedfs_rfmodel.joblib'


In [150]:
joblib.dump(models[7][1], filename_gb_baseline)
joblib.dump(models[10][1], filename_et_baseline)
joblib.dump(models[1][1], filename_rf_baseline)

joblib.dump(fs_models[7][1], filename_gb_fs_baseline)
joblib.dump(fs_models[10][1], filename_et_fs_baseline)
joblib.dump(fs_models[1][1], filename_rf_fs_baseline)

joblib.dump(gb_gs.best_estimator_, filename_gb)
joblib.dump(et_gs.best_estimator_, filename_et)
joblib.dump(rf_gs.best_estimator_, filename_rf)

joblib.dump(gb_gs_fs.best_estimator_, filename_gb_fs)
joblib.dump(et_gs_fs.best_estimator_, filename_et_fs)
joblib.dump(rf_gs_fs.best_estimator_, filename_rf_fs)

AttributeError: 'GradientBoostingClassifier' object has no attribute 'best_estimator_'

###### baseline w original dataset

In [151]:
gb = joblib.load(filename_gb_baseline)
et = joblib.load(filename_et_baseline)
rf = joblib.load(filename_rf_baseline)

###### baseline w fs dataset

In [152]:
gb_fs = joblib.load(filename_gb_fs_baseline)
et_fs = joblib.load(filename_et_fs_baseline)
rf_fs = joblib.load(filename_rf_fs_baseline)

##### tuned w original dataset

In [153]:
gb_gs = joblib.load(filename_gb)
et_gs = joblib.load(filename_et)
rf_gs = joblib.load(filename_rf)

###### tuned w fs dataset

In [154]:
gb_gs_fs = joblib.load(filename_gb_fs)
et_gs_fs = joblib.load(filename_et_fs)
rf_gs_fs = joblib.load(filename_rf_fs)

<hr>

### 12. Voting Classifier

In [164]:
# group / ensemble of best models
estimator = []

estimator.append(('gb', gb))
estimator.append(('et', et))
estimator.append(('rf', rf))
estimator.append(('gb_gs', gb_gs))
estimator.append(('et_gs', et_gs))
estimator.append(('rf_gs', rf_gs))

# Voting Classifier with hard voting
vot_hard = VotingClassifier(estimators = estimator, voting ='hard')
vot_hard.fit(X_train, y_train)
y_pred = vot_hard.predict(X_test)

# using accuracy_score metric to predict accuracy
score = accuracy_score(y_test, y_pred)
print(f'Accuracy Score: {round(score*100,2)}%')

# Voting Classifier with soft voting
#vot_soft = VotingClassifier(estimators = estimator, voting ='soft')
#vot_soft.fit(X_train, y_train)
#y_pred = vot_soft.predict(X_test)

# using accuracy_score
#score = accuracy_score(y_test, y_pred)
#print("Soft Voting Score % d" % score)


Accuracy Score: 90.22%


In [165]:
# group / ensemble of best models
estimator = []

estimator.append(('gb_fs', gb_fs))
estimator.append(('et_fs', et_fs))
estimator.append(('rf_fs', rf_fs))
estimator.append(('gb_gs_fs', gb_gs_fs))
estimator.append(('et_gs_fs', et_gs_fs))
estimator.append(('rf_gs_fs', rf_gs_fs))

# Voting Classifier with hard voting
vot_hard = VotingClassifier(estimators = estimator, voting ='hard')
vot_hard.fit(RF_X_train_importance, y_train)
y_pred = vot_hard.predict(RF_X_test_importance)

# using accuracy_score metric to predict accuracy
score = accuracy_score(y_test, y_pred)
print(f'Accuracy Score:  {round(score*100,2)}%')

# Voting Classifier with soft voting
#vot_soft = VotingClassifier(estimators = estimator, voting ='soft')
#vot_soft.fit(X_train, y_train)
#y_pred = vot_soft.predict(X_test)

# using accuracy_score
#score = accuracy_score(y_test, y_pred)
#print("Soft Voting Score % d" % score)


Accuracy Score:  89.95%
